# 05. Batch Normalization

## 학습 목표
- Batch Normalization의 원리 이해
- Internal Covariate Shift 개념
- 학습/추론 시 동작 차이
- Layer Normalization과의 비교

## 관련 면접 질문
- BatchNorm이 왜 학습을 빠르게 하나요?
- 추론 시 BatchNorm은 어떻게 동작하나요?
- BatchNorm vs LayerNorm 언제 각각 사용하나요?

---
## Pre-Quiz

### Q1. Internal Covariate Shift란?
```
당신의 답: 
```

### Q2. BatchNorm에서 γ, β의 역할은?
```
당신의 답: 
```

### Q3. 추론 시 batch 크기가 1이면 BatchNorm은 어떻게 동작하나요?
```
당신의 답: 
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# matplotlib 한글 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

---
## 1. Batch Normalization 수식

### Forward
$$\mu_B = \frac{1}{m} \sum_{i=1}^{m} x_i$$
$$\sigma^2_B = \frac{1}{m} \sum_{i=1}^{m} (x_i - \mu_B)^2$$
$$\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma^2_B + \epsilon}}$$
$$y_i = \gamma \hat{x}_i + \beta$$

### 핵심 포인트
- 각 **feature(channel)** 별로 정규화
- **batch** 차원에서 mean, variance 계산
- γ, β는 학습 가능한 파라미터

In [ ]:
# =================================================
# TODO: Batch Normalization 구현
# =================================================

class BatchNorm1d:
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        """
        Args:
            num_features: feature 수 (channel 수)
            eps: 분모 0 방지용 작은 값
            momentum: running mean/var 업데이트 비율
        """
        self.eps = eps
        self.momentum = momentum
        
        # 학습 가능 파라미터
        self.gamma = np.ones(num_features)   # scale
        self.beta = np.zeros(num_features)   # shift
        
        # Running statistics (추론 시 사용)
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)
        
        # 학습/추론 모드
        self.training = True
        
        # Backward용 캐시
        self.cache = None
    
    def forward(self, x):
        """
        Args:
            x: (batch, features)
        Returns:
            out: (batch, features)
        """
        # ============================================
        # 여기에 구현하세요
        #
        # training=True:
        #   1. batch mean, var 계산
        #   2. running mean/var 업데이트
        #   3. 정규화: x_hat = (x - mean) / sqrt(var + eps)
        #   4. scale & shift: y = gamma * x_hat + beta
        #   5. cache 저장 (backward용)
        #
        # training=False:
        #   running mean/var 사용
        # ============================================
        pass
    
    def backward(self, dout):
        """
        Args:
            dout: (batch, features)
        Returns:
            dx: (batch, features)
        """
        # ============================================
        # 여기에 구현하세요 (난이도 높음)
        #
        # 힌트: 캐시에서 필요한 값 가져오기
        # x_hat, std = self.cache
        # 
        # dgamma = sum(dout * x_hat, axis=0)
        # dbeta = sum(dout, axis=0)
        # dx_hat = dout * gamma
        # dx = (dx_hat - mean(dx_hat) - x_hat * mean(dx_hat * x_hat)) / std
        # ============================================
        pass
    
    def train(self):
        self.training = True
    
    def eval(self):
        self.training = False

In [ ]:
# ============ 테스트 ============
def test_batch_norm():
    np.random.seed(42)
    
    bn = BatchNorm1d(num_features=4)
    
    # 다양한 scale의 입력
    x = np.random.randn(32, 4) * np.array([1, 10, 100, 1000])
    
    # Training mode
    bn.train()
    out = bn.forward(x)
    
    # 출력이 정규화되었는지 확인 (mean≈0, std≈1)
    out_mean = np.mean(out, axis=0)
    out_std = np.std(out, axis=0)
    
    assert np.allclose(out_mean, 0, atol=1e-6), f"Mean 오류: {out_mean}"
    assert np.allclose(out_std, 1, atol=0.1), f"Std 오류: {out_std}"
    
    print("✅ BatchNorm 테스트 통과!")
    print(f"   Input std per feature: {np.std(x, axis=0)}")
    print(f"   Output mean: {out_mean}")
    print(f"   Output std: {out_std}")

test_batch_norm()

---
## 2. BatchNorm 효과 시각화

In [ ]:
def visualize_batchnorm_effect():
    """BatchNorm 적용 전후 분포 비교"""
    np.random.seed(42)
    
    # 다양한 분포의 데이터
    x1 = np.random.randn(1000) * 5 + 10    # mean=10, std=5
    x2 = np.random.randn(1000) * 0.1 - 3   # mean=-3, std=0.1
    x3 = np.random.randn(1000) * 20        # mean=0, std=20
    
    x = np.stack([x1, x2, x3], axis=1)  # (1000, 3)
    
    bn = BatchNorm1d(3)
    bn.train()
    x_norm = bn.forward(x)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    titles = ['Feature 1 (μ=10, σ=5)', 'Feature 2 (μ=-3, σ=0.1)', 'Feature 3 (μ=0, σ=20)']
    
    for i in range(3):
        # Before
        axes[0, i].hist(x[:, i], bins=50, alpha=0.7)
        axes[0, i].set_title(f'Before BN: {titles[i]}')
        axes[0, i].axvline(x=0, color='r', linestyle='--')
        
        # After
        axes[1, i].hist(x_norm[:, i], bins=50, alpha=0.7, color='green')
        axes[1, i].set_title(f'After BN (μ≈0, σ≈1)')
        axes[1, i].axvline(x=0, color='r', linestyle='--')
        axes[1, i].set_xlim(-4, 4)
    
    plt.tight_layout()
    plt.show()

visualize_batchnorm_effect()

---
## 3. 학습 vs 추론 모드

In [ ]:
# Training vs Inference mode 비교
def compare_train_eval():
    np.random.seed(42)
    
    bn = BatchNorm1d(2)
    
    # 여러 batch로 학습 시뮬레이션
    print("=== Training Phase ===")
    for i in range(5):
        x = np.random.randn(16, 2) * (i + 1) + i  # 점점 다른 분포
        bn.train()
        _ = bn.forward(x)
        print(f"  Batch {i+1}: running_mean = {bn.running_mean}")
    
    print(f"\n최종 running_mean: {bn.running_mean}")
    print(f"최종 running_var: {bn.running_var}")
    
    # 추론 모드
    print("\n=== Inference Phase ===")
    bn.eval()
    
    # 단일 샘플로 테스트 (batch=1)
    x_single = np.array([[5.0, -2.0]])
    out = bn.forward(x_single)
    print(f"  Input: {x_single}")
    print(f"  Output: {out}")
    print("  (running statistics 사용)")

compare_train_eval()

---
## 4. BatchNorm vs LayerNorm 비교

### 정규화 방향의 차이

| 종류 | 정규화 축 | 사용처 |
|------|----------|--------|
| **BatchNorm** | batch 차원 (각 feature별) | CNN, MLP |
| **LayerNorm** | feature 차원 (각 sample별) | Transformer, RNN |

### 시각적 이해 (입력: batch × features)
```
BatchNorm: 세로(↓) 방향으로 정규화
LayerNorm: 가로(→) 방향으로 정규화

       Feature1  Feature2  Feature3
Batch1    ↓         ↓         ↓     ← BatchNorm
Batch2    ↓         ↓         ↓
Batch3    ↓         ↓         ↓
         →→→→→→→→→→→→→→→→→→→→→→→     ← LayerNorm
```

In [ ]:
# =================================================
# TODO: Layer Normalization 구현
# =================================================

class LayerNorm:
    def __init__(self, normalized_shape, eps=1e-5):
        """
        Args:
            normalized_shape: 정규화할 feature 크기
            eps: 분모 0 방지
        """
        self.eps = eps
        self.gamma = np.ones(normalized_shape)
        self.beta = np.zeros(normalized_shape)
    
    def forward(self, x):
        """
        Args:
            x: (batch, features)
        Returns:
            out: (batch, features)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. 각 sample별로 mean, var 계산 (axis=-1)
        # 2. 정규화
        # 3. scale & shift
        # ============================================
        pass

In [ ]:
# BatchNorm vs LayerNorm 시각화
def compare_bn_ln():
    np.random.seed(42)
    
    x = np.random.randn(4, 8) * np.array([[1], [5], [10], [20]])  # 각 batch마다 다른 scale
    
    bn = BatchNorm1d(8)
    ln = LayerNorm(8)
    
    bn.train()
    x_bn = bn.forward(x)
    x_ln = ln.forward(x)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    im0 = axes[0].imshow(x, cmap='RdBu', aspect='auto')
    axes[0].set_title('Original')
    axes[0].set_xlabel('Features')
    axes[0].set_ylabel('Batch')
    plt.colorbar(im0, ax=axes[0])
    
    im1 = axes[1].imshow(x_bn, cmap='RdBu', aspect='auto', vmin=-2, vmax=2)
    axes[1].set_title('BatchNorm (↓ feature별)')
    axes[1].set_xlabel('Features')
    axes[1].set_ylabel('Batch')
    plt.colorbar(im1, ax=axes[1])
    
    im2 = axes[2].imshow(x_ln, cmap='RdBu', aspect='auto', vmin=-2, vmax=2)
    axes[2].set_title('LayerNorm (→ sample별)')
    axes[2].set_xlabel('Features')
    axes[2].set_ylabel('Batch')
    plt.colorbar(im2, ax=axes[2])
    
    plt.tight_layout()
    plt.show()
    
    # 통계 출력
    print("BatchNorm: 각 feature(column)의 mean≈0, std≈1")
    print(f"  Column means: {x_bn.mean(axis=0)[:4].round(3)}...")
    print(f"  Column stds:  {x_bn.std(axis=0)[:4].round(3)}...")
    
    print("\nLayerNorm: 각 sample(row)의 mean≈0, std≈1")
    print(f"  Row means: {x_ln.mean(axis=1).round(3)}")
    print(f"  Row stds:  {x_ln.std(axis=1).round(3)}")

compare_bn_ln()

---
## 5. 언제 어떤 Normalization을 사용할까?

| 상황 | 추천 | 이유 |
|------|------|------|
| CNN (이미지) | BatchNorm | 각 feature map의 분포 정규화 효과적 |
| Transformer | LayerNorm | 가변 길이 시퀀스, batch 독립성 필요 |
| RNN | LayerNorm | 시퀀스 길이 변화, batch 통계 불안정 |
| 작은 batch | LayerNorm/GroupNorm | BatchNorm은 작은 batch에서 불안정 |
| 추론 시 batch=1 | LayerNorm 유리 | BatchNorm은 running stats 의존 |

---
## Post-Quiz

### Q1. BatchNorm의 γ, β를 학습하는 이유는?
```
당신의 답: 
```

### Q2. Transformer에서 BatchNorm 대신 LayerNorm을 쓰는 이유는?
```
당신의 답: 
```

### Q3. BatchNorm이 정규화 효과(regularization)를 갖는 이유는?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class BatchNorm1d:
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        
        self.gamma = np.ones(num_features)
        self.beta = np.zeros(num_features)
        
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)
        
        self.training = True
        self.cache = None
    
    def forward(self, x):
        if self.training:
            mean = np.mean(x, axis=0)
            var = np.var(x, axis=0)
            
            # Running stats 업데이트
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
            
            std = np.sqrt(var + self.eps)
            x_hat = (x - mean) / std
            
            self.cache = (x_hat, std)
        else:
            std = np.sqrt(self.running_var + self.eps)
            x_hat = (x - self.running_mean) / std
        
        return self.gamma * x_hat + self.beta
    
    def backward(self, dout):
        x_hat, std = self.cache
        m = dout.shape[0]
        
        self.dgamma = np.sum(dout * x_hat, axis=0)
        self.dbeta = np.sum(dout, axis=0)
        
        dx_hat = dout * self.gamma
        dx = (dx_hat - np.mean(dx_hat, axis=0) - x_hat * np.mean(dx_hat * x_hat, axis=0)) / std
        
        return dx


class LayerNorm:
    def __init__(self, normalized_shape, eps=1e-5):
        self.eps = eps
        self.gamma = np.ones(normalized_shape)
        self.beta = np.zeros(normalized_shape)
    
    def forward(self, x):
        mean = np.mean(x, axis=-1, keepdims=True)
        var = np.var(x, axis=-1, keepdims=True)
        x_hat = (x - mean) / np.sqrt(var + self.eps)
        return self.gamma * x_hat + self.beta
```

</details>

<details>
<summary>👉 Quiz 정답</summary>

### Pre-Quiz

**Q1. Internal Covariate Shift**
학습 중 이전 레이어의 파라미터가 변하면서 현재 레이어의 입력 분포가 계속 바뀌는 현상. 각 레이어가 새로운 분포에 적응해야 해서 학습이 느려짐.

**Q2. γ, β의 역할**
정규화 후 표현력 복구. 항상 mean=0, std=1이면 표현력이 제한될 수 있음. 모델이 필요하면 원래 분포로 돌릴 수 있도록 scale(γ)과 shift(β)를 학습.

**Q3. 추론 시 batch=1**
학습 중 저장한 running mean/var를 사용. 현재 batch의 통계가 아닌 학습 전체의 통계를 사용하므로 batch 크기와 무관하게 동작.

### Post-Quiz

**Q1. γ, β 학습 이유**
항등 변환(identity)을 포함하도록. γ=std, β=mean이면 정규화 전으로 복원됨. 모델이 BN이 필요한 정도를 스스로 학습.

**Q2. Transformer에서 LayerNorm 이유**
1) 시퀀스 길이가 가변적 2) batch 내 샘플들이 독립적이어야 함 3) 작은 batch에서도 안정적 4) 추론 시 batch=1도 자연스럽게 처리

**Q3. BatchNorm의 정규화 효과**
미니배치 통계의 노이즈가 암묵적 정규화 역할. 매 iteration마다 다른 batch 샘플 → 다른 mean/var → 학습에 노이즈 추가 (dropout과 유사한 효과).

</details>